In [42]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/llama-2-13b-bnb-4bit",
#     "unsloth/codellama-34b-bnb-4bit",
#     "unsloth/tinyllama-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
#     "unsloth/gemma-2b-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/jianing/Github/LLMRL/checkpoints/tinyllama-MetaMath", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
tokenizer.padding_side = "left" # Padding side for faster inference

#well for llama3 'pre-trained models usually do not stop completions naturally.'
#https://github.com/meta-llama/llama3/blob/main/example_text_completion.py

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.641 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [55]:
# alpaca_prompt = Copied from above
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    "Eliza's rate per hour for the first 40 hours she works each week is $10. She also receives an overtime pay of 1.2 times her regular hourly rate. If Eliza worked for 45 hours this week, how much are her earnings for this week?"
], return_tensors = "pt").to("cuda")

_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256, do_sample=True, temperature=0.2, top_k=40, eos_token_id = tokenizer.eos_token_id)



<s> Eliza's rate per hour for the first 40 hours she works each week is $10. She also receives an overtime pay of 1.2 times her regular hourly rate. If Eliza worked for 45 hours this week, how much are her earnings for this week? Eliza's regular hourly rate is $10, so for the first 40 hours she works, she earns 40 * $10 = $400.
For the overtime hours, she earns 1.2 * $10 = $12 per hour.
For the overtime hours, she works 45 - 40 = 5 hours.
So, her earnings for the overtime hours is 5 * $12 = $60.
Therefore, her total earnings for the week is $400 + $60 = $460.
#### 460
The answer is: 460</s>


In [41]:
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Eliza's rate per hour for the first 40 hours she works each week is $10. She also receives an overtime pay of 1.2 times her regular hourly rate. If Eliza worked for 45 hours this week, how much are her earnings for this week? Eliza's regular hourly rate is $10, so for the first 40 hours she works, she earns 40 * $10 = $400.\nFor the overtime hours, she earns 1.2 times her regular hourly rate, so she earns 1.2 * $10 = $12 per hour.\nFor the overtime hours, she earns $12 * 45 = $520.\nIn total, Eliza earns $400 + $520 = $920 for this week.\n#### 920\nThe answer is: 920</s>"]

checkpoint 2000 is the best